In [301]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MultiLabelBinarizer
from datetime import datetime


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

In [302]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

# Setup

In [303]:
VERSION    = 11
#DIR_TRAIN  = '../input/sf-autoru-solve-v4/' # подключил к ноутбуку свой внешний датасет
#DIR_TEST   = '../input/sf-dst-car-price/'
VAL_SIZE   = 0.33   # 33%
N_FOLDS    = 5

# CATBOOST
ITERATIONS = 2000
LR         = 0.1

# Data

In [304]:
#cars = pd.read_csv('united_3.csv')
cars = pd.read_csv('cars_new.csv')
#cars.drop(columns=['Unnamed: 0'], inplace=True)
cars.head()

,test,style,body,color,fuel,model_date,doors,production_date,transmission,engine_displacement,engine_power,mileage,drive_type,wheel,condition,owners,vehicle_title,features,price,tax,tax_per_horse,days_in_use,mileage_per_day
0,0,520i,body 1,color 3,бензин,model date 6,4,1991,механическая,eng dis 2,129,300000.0,задний,Левый,Не требует ремонта,3,Оригинал,Unknown,60000.0,4515.0,35,10797,27.785496
1,0,316i,body 1,color 1,бензин,model date 7,4,1992,механическая,eng dis 4,102,300000.0,задний,Левый,Не требует ремонта,3,Дубликат,Unknown,68000.0,2550.0,25,10432,28.757669
2,0,318i,body 1,color 5,бензин,model date 7,4,1994,механическая,eng dis 5,115,350000.0,задний,Левый,Не требует ремонта,3,Дубликат,Unknown,70000.0,2875.0,25,9701,36.078755
3,0,520i,body 1,color 1,бензин,model date 6,4,1992,механическая,eng dis 2,150,100000.0,задний,Левый,Не требует ремонта,3,Оригинал,Unknown,70000.0,5250.0,35,10432,9.585890
4,0,520i,body 1,color 1,бензин,model date 6,4,1982,механическая,eng dis 2,125,250000.0,задний,Левый,Не требует ремонта,1,Оригинал,Unknown,70000.0,3125.0,25,14084,17.750639


In [305]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9961 entries, 0 to 9960
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   test                 9961 non-null   int64  
 1   style                9849 non-null   object 
 2   body                 9961 non-null   object 
 3   color                9961 non-null   object 
 4   fuel                 9961 non-null   object 
 5   model_date           9961 non-null   object 
 6   doors                9961 non-null   int64  
 7   production_date      9961 non-null   int64  
 8   transmission         9961 non-null   object 
 9   engine_displacement  9960 non-null   object 
 10  engine_power         9961 non-null   int64  
 11  mileage              9961 non-null   float64
 12  drive_type           9961 non-null   object 
 13  wheel                9961 non-null   object 
 14  condition            9961 non-null   object 
 15  owners               9961 non-null   i

In [308]:
cars['style'] = cars.apply(lambda x: 'Unknown' if pd.isna(x.style) else x.style, axis=1)
cars['engine_displacement'] = cars.apply(lambda x: 'eng dis 1' if pd.isna(x.engine_displacement) else x.engine_displacement, axis=1)

In [309]:
#перевод категориальных str в int, для методов которым на вход необходимо int.
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

le = LabelEncoder()

le.fit(cars['style'])
cars['style'] = le.transform(cars['style'])

le.fit(cars['body'])
cars['body'] = le.transform(cars['body'])


le.fit(cars['color'])
cars['color'] = le.transform(cars['color'])

le.fit(cars['fuel'])
cars['fuel'] = le.transform(cars['fuel'])

le.fit(cars['model_date'])
cars['model_date'] = le.transform(cars['model_date'])

le.fit(cars['transmission'])
cars['transmission'] = le.transform(cars['transmission'])

le.fit(cars['engine_displacement'])
cars['engine_displacement'] = le.transform(cars['engine_displacement'])

le.fit(cars['drive_type'])
cars['drive_type'] = le.transform(cars['drive_type'])

le.fit(cars['wheel'])
cars['wheel'] = le.transform(cars['wheel'])

le.fit(cars['condition'])
cars['condition'] = le.transform(cars['condition'])

le.fit(cars['vehicle_title'])
cars['vehicle_title'] = le.transform(cars['vehicle_title'])

In [310]:
train = cars[cars['test'] == 0]
train.reset_index(drop=True)
test = cars[cars['test'] == 1]
test.reset_index(drop=True)
sample_submission = pd.read_csv('sample_submission.csv')

# Data Preprocessing

In [311]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    #df_output.drop(['Таможня', 'Состояние', 'id'], axis=1, inplace=True,)
    
    
    # ################### Feature Engineering ####################################################
    df_output['years_in_use'] = df_output['production_date'].apply(lambda x: datetime.now().year-x) #количество лет использования машины
    
    df_output['mileage_per_day'] = df_output['mileage'] / (df_output['years_in_use'] * 365) #пробег машины в день
    df_output['mileage_per_day'] = df_output.apply(lambda x: x.mileage_per_day if x.years_in_use !=0 else 0, axis=1)
    
    # Переведем комплектацию в dummy-переменные
    #df_output['features'] = df_output.apply(lambda x: '[Unknown]' if x.features == 'Unknown' else x.features, axis=1)
    #df_output['features'] = df_output['features'].apply(lambda x: str(x)[1:-1])
    #df_output['features'] = df_output['features'].apply(lambda x: x.replace("'", ""))
    #features = df_output['features'].str.get_dummies(sep=',')
    #df_output = df_output.join(features)
    
    # ################### fix ############################################################## 
    # Переводим признаки из float в int (иначе catboost выдает ошибку)
    for feature in ['engine_power', 'mileage', 'production_date', 'owners','tax', 'price', 'mileage_per_day']:
        df_output[feature]=df_output[feature].astype('int32')
    #for feature in ['mileage', 'price', 'tax', 'mileage_per_day']:
        #df_output[feature]=df_output[feature].astype('int32')
    
    
    # ################### Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    #df_output.drop(['description', 'features', 'test'], axis=1, inplace=True)
    df_output.drop(['features'], axis=1, inplace=True)
    
    return df_output

In [312]:
train_preproc = preproc_data(train)
X_sub = preproc_data(test)

In [313]:
X = train_preproc.drop(['price'], axis=1)
y = train_preproc.price.values
X_sub = X_sub.drop(['price'], axis=1)

# Train Split

In [314]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

# CatBoost

In [315]:
# чтобы не писать весь список этих признаков, просто вывел их через nunique(). и так сойдет)
X_train.nunique()

test                      1
style                   156
body                      4
color                     6
fuel                      4
model_date                9
doors                     4
production_date          35
transmission              3
engine_displacement       5
engine_power             66
mileage                1605
drive_type                3
wheel                     1
condition                 1
owners                    3
vehicle_title             2
tax                      66
tax_per_horse             8
days_in_use              35
mileage_per_day         149
years_in_use             35
dtype: int64

In [316]:
# Keep list of all categorical features in dataset to specify this for CatBoost
cat_features_ids = np.where(X_train.apply(pd.Series.nunique) < 3000)[0].tolist()

# Fit

In [317]:
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
model.fit(X_train, y_train,
         cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 1.0122368	test: 1.0900439	best: 1.0900439 (0)	total: 58.1ms	remaining: 1m 56s
100:	learn: 0.1747570	test: 0.1742578	best: 0.1742429 (99)	total: 7.18s	remaining: 2m 15s
200:	learn: 0.1492527	test: 0.1608371	best: 0.1607240 (197)	total: 16s	remaining: 2m 23s
300:	learn: 0.1363307	test: 0.1568491	best: 0.1565189 (292)	total: 24.6s	remaining: 2m 18s
400:	learn: 0.1274770	test: 0.1541592	best: 0.1540924 (390)	total: 33.8s	remaining: 2m 14s
500:	learn: 0.1206797	test: 0.1532466	best: 0.1532466 (500)	total: 43.2s	remaining: 2m 9s

600:	learn: 0.1145650	test: 0.1524003	best: 0.1522308 (566)	total: 52.4s	remaining: 2m 1s
700:	learn: 0.1092361	test: 0.1520521	best: 0.1520035 (685)	total: 1m 1s	remaining: 1m 54s
800:	learn: 0.1038886	test: 0.1518795	best: 0.1514927 (729)	total: 1m 10s	remaining: 1m 45s
900:	learn: 0.0989025	test: 0.1522278	best: 0.1514927 (729)	total: 1m 19s	remaining: 1m 37s
1000:	learn: 0.0949720	test: 0.1527204	best: 0.1514927 (729)	total: 1m 29s	remaining: 1m 29s
11

In [318]:
X_sub.reset_index(drop=True)

,test,style,body,color,fuel,model_date,doors,production_date,transmission,engine_displacement,engine_power,mileage,drive_type,wheel,condition,owners,vehicle_title,tax,tax_per_horse,days_in_use,mileage_per_day,years_in_use
0,1,80,0,0,2,2,4,2017,0,1,190,158836,0,0,0,1,1,9500,50,1300,145,3
1,1,34,0,0,2,0,4,2019,0,1,150,10,0,0,0,1,1,5250,35,570,0,1
2,1,107,0,0,0,4,4,2012,0,0,407,120000,2,0,0,2,1,61050,150,3127,41,8
3,1,28,1,0,2,2,5,2015,0,0,249,111466,2,0,0,2,1,18675,75,2031,61,5
4,1,156,1,0,2,2,5,2019,0,0,381,11891,2,0,0,1,1,57150,150,570,32,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3832,1,82,0,0,0,1,4,2013,0,1,184,98000,0,0,0,3,1,9200,50,2761,38,7
3833,1,28,1,0,2,1,5,2016,0,0,249,169615,2,0,0,1,1,18675,75,1666,116,4
3834,1,88,0,0,0,5,4,2006,0,3,218,180000,2,0,0,3,1,14170,65,5318,35,14
3835,1,91,0,0,0,4,4,2012,0,1,245,137500,2,0,0,3,1,18375,75,3127,47,8


In [319]:
model.save_model('catboost_single_model_baseline.model')

# Submission

In [320]:
predict_submission = model.predict(X_sub)
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_v{VERSION}.csv', index=False)
sample_submission.head(10)

,id,price
0,0,1.889122e+06
1,1,2.950978e+06
2,2,1.308150e+06
3,3,2.608926e+06
4,4,5.590183e+06
5,5,2.391322e+06
6,6,1.220398e+06
7,7,7.059896e+05
8,8,1.591438e+06
9,9,1.731920e+06


# CV

In [322]:
train.tax_per_horse.value_counts()

150    1767
50     1488
75     1148
35      652
65      501
45      340
25      222
12        6
Name: tax_per_horse, dtype: int64

Перед тем как бежать перебирать признаки и модели, поговорим о кросс-валидации (CV)

Когда мы делаем отбор признаков или перебираем настройки модели, мы постоянно смотрим в тестовые данные (X_test), что может привести к подгону под тестовые данные. В итоге мы получим Переобучение (overfitting).
Чтобы избежать этого, можно сразу использовать кросс-валидацию по фолдам (подробнее в модуле Классический Machine Learning >Модуль 7. Валидация данных >7.2. Разбиение выборки). 

Ниже представлен Пример, как можно организовать обучение модели на 5 фолдах, с дальнейшим объединением предсказаний от каждой модели.

In [323]:
def cat_model(y_train, X_train, X_test, y_test):
    model = CatBoostRegressor(iterations = ITERATIONS,
                              learning_rate = LR,
                              eval_metric='MAPE',
                              random_seed = RANDOM_SEED,)
    model.fit(X_train, y_train,
              cat_features=cat_features_ids,
              eval_set=(X_test, y_test),
              verbose=False,
              use_best_model=True,
              plot=False)
    
    return(model)


def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

submissions = pd.DataFrame(0,columns=["sub_1"], index=sample_submission.index) # куда пишем предикты по каждой модели
score_ls = []
splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))

for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=N_FOLDS,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y[train_idx], X.iloc[test_idx], y[test_idx]
    # model for this fold
    model = cat_model(y_train, X_train, X_test, y_test,)
    # score model on test
    test_predict = model.predict(X_test)
    test_score = mape(y_test, test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {mape(y_test, test_predict):0.3f}")
    # submissions
    submissions[f'sub_{idx+1}'] = model.predict(X_sub)
    model.save_model(f'catboost_fold_{idx+1}.model')
    
print(f'Mean Score: {np.mean(score_ls):0.3f}')
print(f'Std Score: {np.std(score_ls):0.4f}')
print(f'Max Score: {np.max(score_ls):0.3f}')
print(f'Min Score: {np.min(score_ls):0.3f}')

1 Fold Test MAPE: 0.156
2 Fold Test MAPE: 0.152
3 Fold Test MAPE: 0.140
4 Fold Test MAPE: 0.146
5 Fold Test MAPE: 0.145

Mean Score: 0.148
Std Score: 0.0054
Max Score: 0.156
Min Score: 0.140


In [324]:
sample_submission = pd.read_csv('sample_submission.csv')
submissions['blend'] = (submissions.sum(axis=1))/len(submissions.columns)
sample_submission['price'] = submissions['blend'].values
sample_submission.head(10)
sample_submission.price = round(sample_submission.price/10000,0)*10000*0.92
sample_submission.to_csv(f'submission_blend_v3.csv', index=False)
sample_submission.head(10)

,id,price
0,0,1720400.0
1,1,2576000.0
2,2,1168400.0
3,3,2456400.0
4,4,5096800.0
5,5,2116000.0
6,6,1021200.0
7,7,671600.0
8,8,1453600.0
9,9,1462800.0


## Stacking

Давайте сначала разберемся, что-же такое этот Stacking.

Начнем с Ансамбля моделей:
Допустим, вы обучили Различные модели. Теперь мы можем просто объединить их предсказания и получить средневзвешенное предсказание по всем моделям. При этом, чем разнообразней модели - тем лучше результат мы получим. 

А теперь Stacking:
У нас есть предсказания от разных моделей, почему бы не использовать их как новые признаки/фитчи и не построить поверх этих предсказаний новую модель? Это основная идея Stacking-a.

Далее его еще можно бесконечно усложнять. Например, добавляя модели обученные на разных выборках и/или с разным составом признаков (bagging), или увеличивая уровни стекинга.

In [325]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from collections import defaultdict


def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R-sq = {r2:.2f}, MAPE = {mape:.2f} ')

In [326]:
from sklearn.base import clone
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """
    Computes meta-features usinf the classifier cls
    
    :arg model: scikit-learn classifier
    :arg X_train, y_train: training set
    :arg X_test: testing set
    :arg cv: cross-validation folding
    """
    
    X_meta_train = np.zeros_like(y_train, dtype = np.float32)
    X_meta_test = np.zeros(len(X_test), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict(X_fold_predict)
        
        print_regression_metrics(X_meta_train[predict_fold_index], y_train[predict_fold_index])
        X_meta_test += folded_clf.predict(X_test)
    
    n = cv.n_splits
    X_meta_test = X_meta_test / n
    
    return X_meta_train, X_meta_test

In [327]:
def generate_meta_features(regressors, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(regressors)
    ]
    
    stacked_features_train = np.stack([
        features_train for features_train, features_test in features
    ], axis=-1)

    stacked_features_test = np.stack([
        features_test for features_train, features_test in features
    ], axis=-1)
    
    return stacked_features_train, stacked_features_test

In [329]:
#X = train_preproc.drop(['price'], axis=1)
#y = train_preproc.price.values
#X_sub = X_sub.drop(['price'], axis=1)

#X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size = VAL_SIZE, random_state=RANDOM_SEED)

cv = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

def compute_metric(clf, X_train=X_train, y_train=y_train, X_test=X_test):
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return print_regression_metrics(y_test, y_test_pred)

In [330]:
from sklearn.preprocessing import StandardScaler
# Стандартизируем данные:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
X_sub = scaler.fit_transform(X_sub)

stacked_features_train, stacked_features_test = generate_meta_features([
    RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED),
    BaggingRegressor(ExtraTreesRegressor(n_estimators=100, random_state=RANDOM_SEED)),
    CatBoostRegressor(loss_function = 'MAE',
                         eval_metric = 'MAPE',
                         learning_rate=0.005,
                         iterations=4500,
                         l2_leaf_reg=2,
                         depth=6,
                         bootstrap_type = 'Bayesian',
                         random_seed=42,
                         od_type='Iter',
                         od_wait=100)
    ], X_train, X_sub, y_train, cv)


#Строим мета-алгоритм

final_model = LinearRegression()
final_model.fit(stacked_features_train, y_train)

compute_metric(final_model, X_train=stack_future_train, X_test=stack_future_test, y_train=y_train)

RMSE = 310813.57, MAE = 161032.78, R-sq = 0.93, MAPE = 0.15 
RMSE = 352077.19, MAE = 160344.29, R-sq = 0.89, MAPE = 0.14 
RMSE = 306962.14, MAE = 159216.87, R-sq = 0.93, MAPE = 0.14 
RMSE = 296751.70, MAE = 158933.81, R-sq = 0.93, MAPE = 0.14 
RMSE = 283347.90, MAE = 154179.94, R-sq = 0.94, MAPE = 0.14 
RMSE = 362035.39, MAE = 164078.59, R-sq = 0.90, MAPE = 0.15 
RMSE = 351863.29, MAE = 157761.27, R-sq = 0.88, MAPE = 0.14 
RMSE = 320748.51, MAE = 156133.95, R-sq = 0.92, MAPE = 0.13 
RMSE = 266726.96, MAE = 153435.86, R-sq = 0.94, MAPE = 0.14 
RMSE = 251185.77, MAE = 144373.13, R-sq = 0.95, MAPE = 0.13 
0:	learn: 0.7971781	total: 12.3ms	remaining: 55.5s
1:	learn: 0.7936466	total: 20.3ms	remaining: 45.6s
2:	learn: 0.7904774	total: 28ms	remaining: 41.9s
3:	learn: 0.7873264	total: 35.3ms	remaining: 39.7s
4:	learn: 0.7842696	total: 42.6ms	remaining: 38.3s
5:	learn: 0.7810507	total: 49.8ms	remaining: 37.3s
6:	learn: 0.7776422	total: 58.3ms	remaining: 37.4s
7:	learn: 0.7744035	total: 65.6ms	r

151:	learn: 0.4429564	total: 858ms	remaining: 24.5s
152:	learn: 0.4415104	total: 863ms	remaining: 24.5s
153:	learn: 0.4400364	total: 867ms	remaining: 24.5s
154:	learn: 0.4386006	total: 872ms	remaining: 24.4s
155:	learn: 0.4371159	total: 877ms	remaining: 24.4s
156:	learn: 0.4355957	total: 882ms	remaining: 24.4s
157:	learn: 0.4341662	total: 887ms	remaining: 24.4s
158:	learn: 0.4326164	total: 892ms	remaining: 24.4s
159:	learn: 0.4311407	total: 898ms	remaining: 24.4s
160:	learn: 0.4297146	total: 903ms	remaining: 24.3s
161:	learn: 0.4282181	total: 910ms	remaining: 24.4s
162:	learn: 0.4266786	total: 915ms	remaining: 24.3s
163:	learn: 0.4252338	total: 920ms	remaining: 24.3s
164:	learn: 0.4238833	total: 925ms	remaining: 24.3s
165:	learn: 0.4223757	total: 930ms	remaining: 24.3s
166:	learn: 0.4211419	total: 935ms	remaining: 24.3s
167:	learn: 0.4196953	total: 940ms	remaining: 24.2s
168:	learn: 0.4184233	total: 945ms	remaining: 24.2s
169:	learn: 0.4169439	total: 950ms	remaining: 24.2s
170:	learn: 

313:	learn: 0.2714040	total: 1.7s	remaining: 22.6s
314:	learn: 0.2706611	total: 1.7s	remaining: 22.6s
315:	learn: 0.2699246	total: 1.71s	remaining: 22.6s
316:	learn: 0.2692770	total: 1.71s	remaining: 22.6s
317:	learn: 0.2686613	total: 1.72s	remaining: 22.6s
318:	learn: 0.2679615	total: 1.72s	remaining: 22.6s
319:	learn: 0.2672349	total: 1.73s	remaining: 22.6s
320:	learn: 0.2664968	total: 1.73s	remaining: 22.6s
321:	learn: 0.2659186	total: 1.74s	remaining: 22.5s
322:	learn: 0.2652259	total: 1.74s	remaining: 22.5s
323:	learn: 0.2645818	total: 1.75s	remaining: 22.5s
324:	learn: 0.2639259	total: 1.75s	remaining: 22.5s
325:	learn: 0.2632882	total: 1.76s	remaining: 22.5s
326:	learn: 0.2625532	total: 1.76s	remaining: 22.5s
327:	learn: 0.2618378	total: 1.77s	remaining: 22.5s
328:	learn: 0.2612973	total: 1.77s	remaining: 22.5s
329:	learn: 0.2606483	total: 1.78s	remaining: 22.5s
330:	learn: 0.2599571	total: 1.78s	remaining: 22.5s
331:	learn: 0.2592489	total: 1.79s	remaining: 22.5s
332:	learn: 0.

475:	learn: 0.1940938	total: 2.54s	remaining: 21.5s
476:	learn: 0.1937644	total: 2.55s	remaining: 21.5s
477:	learn: 0.1934470	total: 2.55s	remaining: 21.5s
478:	learn: 0.1931414	total: 2.56s	remaining: 21.5s
479:	learn: 0.1928637	total: 2.56s	remaining: 21.5s
480:	learn: 0.1925477	total: 2.57s	remaining: 21.4s
481:	learn: 0.1922932	total: 2.57s	remaining: 21.4s
482:	learn: 0.1919840	total: 2.58s	remaining: 21.4s
483:	learn: 0.1916700	total: 2.58s	remaining: 21.4s
484:	learn: 0.1913626	total: 2.59s	remaining: 21.4s
485:	learn: 0.1910633	total: 2.59s	remaining: 21.4s
486:	learn: 0.1907620	total: 2.6s	remaining: 21.4s
487:	learn: 0.1904772	total: 2.6s	remaining: 21.4s
488:	learn: 0.1901854	total: 2.61s	remaining: 21.4s
489:	learn: 0.1898696	total: 2.61s	remaining: 21.4s
490:	learn: 0.1895777	total: 2.62s	remaining: 21.4s
491:	learn: 0.1893030	total: 2.62s	remaining: 21.4s
492:	learn: 0.1890268	total: 2.63s	remaining: 21.4s
493:	learn: 0.1887617	total: 2.63s	remaining: 21.4s
494:	learn: 0.

637:	learn: 0.1603895	total: 3.38s	remaining: 20.5s
638:	learn: 0.1602557	total: 3.39s	remaining: 20.5s
639:	learn: 0.1601301	total: 3.39s	remaining: 20.5s
640:	learn: 0.1599763	total: 3.4s	remaining: 20.5s
641:	learn: 0.1598344	total: 3.4s	remaining: 20.5s
642:	learn: 0.1596812	total: 3.41s	remaining: 20.5s
643:	learn: 0.1595598	total: 3.42s	remaining: 20.5s
644:	learn: 0.1594200	total: 3.42s	remaining: 20.4s
645:	learn: 0.1593082	total: 3.42s	remaining: 20.4s
646:	learn: 0.1591495	total: 3.43s	remaining: 20.4s
647:	learn: 0.1590209	total: 3.44s	remaining: 20.4s
648:	learn: 0.1588780	total: 3.44s	remaining: 20.4s
649:	learn: 0.1587603	total: 3.45s	remaining: 20.4s
650:	learn: 0.1586379	total: 3.45s	remaining: 20.4s
651:	learn: 0.1585187	total: 3.46s	remaining: 20.4s
652:	learn: 0.1583832	total: 3.46s	remaining: 20.4s
653:	learn: 0.1582704	total: 3.47s	remaining: 20.4s
654:	learn: 0.1581527	total: 3.47s	remaining: 20.4s
655:	learn: 0.1579970	total: 3.48s	remaining: 20.4s
656:	learn: 0.

...

3803:	learn: 0.1052758	total: 20.5s	remaining: 3.75s
3804:	learn: 0.1052703	total: 20.5s	remaining: 3.75s
3805:	learn: 0.1052674	total: 20.5s	remaining: 3.74s
3806:	learn: 0.1052616	total: 20.5s	remaining: 3.74s
3807:	learn: 0.1052549	total: 20.5s	remaining: 3.73s
3808:	learn: 0.1052506	total: 20.5s	remaining: 3.73s
3809:	learn: 0.1052451	total: 20.5s	remaining: 3.72s
3810:	learn: 0.1052428	total: 20.6s	remaining: 3.71s
3811:	learn: 0.1052391	total: 20.6s	remaining: 3.71s
3812:	learn: 0.1052348	total: 20.6s	remaining: 3.7s
3813:	learn: 0.1052326	total: 20.6s	remaining: 3.7s
3814:	learn: 0.1052238	total: 20.6s	remaining: 3.69s
3815:	learn: 0.1052201	total: 20.6s	remaining: 3.69s
3816:	learn: 0.1052162	total: 20.6s	remaining: 3.68s
3817:	learn: 0.1052146	total: 20.6s	remaining: 3.68s
3818:	learn: 0.1052101	total: 20.6s	remaining: 3.67s
3819:	learn: 0.1051999	total: 20.6s	remaining: 3.67s
3820:	learn: 0.1051979	total: 20.6s	remaining: 3.66s
3821:	learn: 0.1051937	total: 20.6s	remaining: 3

3960:	learn: 0.1045037	total: 21.4s	remaining: 2.9s
3961:	learn: 0.1045032	total: 21.4s	remaining: 2.9s
3962:	learn: 0.1045014	total: 21.4s	remaining: 2.89s
3963:	learn: 0.1044962	total: 21.4s	remaining: 2.89s
3964:	learn: 0.1044905	total: 21.4s	remaining: 2.88s
3965:	learn: 0.1044883	total: 21.4s	remaining: 2.88s
3966:	learn: 0.1044864	total: 21.4s	remaining: 2.87s
3967:	learn: 0.1044856	total: 21.4s	remaining: 2.87s
3968:	learn: 0.1044820	total: 21.4s	remaining: 2.86s
3969:	learn: 0.1044788	total: 21.4s	remaining: 2.86s
3970:	learn: 0.1044774	total: 21.4s	remaining: 2.85s
3971:	learn: 0.1044756	total: 21.4s	remaining: 2.85s
3972:	learn: 0.1044722	total: 21.4s	remaining: 2.84s
3973:	learn: 0.1044651	total: 21.4s	remaining: 2.83s
3974:	learn: 0.1044598	total: 21.4s	remaining: 2.83s
3975:	learn: 0.1044525	total: 21.4s	remaining: 2.82s
3976:	learn: 0.1044453	total: 21.4s	remaining: 2.82s
3977:	learn: 0.1044414	total: 21.4s	remaining: 2.81s
3978:	learn: 0.1044319	total: 21.4s	remaining: 2

4140:	learn: 0.1037175	total: 22.4s	remaining: 1.94s
4141:	learn: 0.1037136	total: 22.4s	remaining: 1.93s
4142:	learn: 0.1037068	total: 22.4s	remaining: 1.93s
4143:	learn: 0.1036992	total: 22.4s	remaining: 1.92s
4144:	learn: 0.1036964	total: 22.4s	remaining: 1.92s
4145:	learn: 0.1036929	total: 22.4s	remaining: 1.91s
4146:	learn: 0.1036916	total: 22.4s	remaining: 1.91s
4147:	learn: 0.1036852	total: 22.4s	remaining: 1.9s
4148:	learn: 0.1036794	total: 22.4s	remaining: 1.9s
4149:	learn: 0.1036756	total: 22.4s	remaining: 1.89s
4150:	learn: 0.1036734	total: 22.4s	remaining: 1.88s
4151:	learn: 0.1036713	total: 22.4s	remaining: 1.88s
4152:	learn: 0.1036693	total: 22.4s	remaining: 1.87s
4153:	learn: 0.1036622	total: 22.4s	remaining: 1.87s
4154:	learn: 0.1036572	total: 22.4s	remaining: 1.86s
4155:	learn: 0.1036550	total: 22.4s	remaining: 1.86s
4156:	learn: 0.1036518	total: 22.4s	remaining: 1.85s
4157:	learn: 0.1036476	total: 22.5s	remaining: 1.85s
4158:	learn: 0.1036451	total: 22.5s	remaining: 1

4317:	learn: 0.1029568	total: 23.3s	remaining: 984ms
4318:	learn: 0.1029515	total: 23.3s	remaining: 979ms
4319:	learn: 0.1029461	total: 23.4s	remaining: 973ms
4320:	learn: 0.1029426	total: 23.4s	remaining: 968ms
4321:	learn: 0.1029356	total: 23.4s	remaining: 962ms
4322:	learn: 0.1029290	total: 23.4s	remaining: 957ms
4323:	learn: 0.1029232	total: 23.4s	remaining: 952ms
4324:	learn: 0.1029156	total: 23.4s	remaining: 946ms
4325:	learn: 0.1029138	total: 23.4s	remaining: 941ms
4326:	learn: 0.1029082	total: 23.4s	remaining: 935ms
4327:	learn: 0.1029046	total: 23.4s	remaining: 930ms
4328:	learn: 0.1029015	total: 23.4s	remaining: 925ms
4329:	learn: 0.1029014	total: 23.4s	remaining: 919ms
4330:	learn: 0.1028963	total: 23.4s	remaining: 914ms
4331:	learn: 0.1028889	total: 23.4s	remaining: 908ms
4332:	learn: 0.1028849	total: 23.4s	remaining: 903ms
4333:	learn: 0.1028779	total: 23.4s	remaining: 898ms
4334:	learn: 0.1028732	total: 23.4s	remaining: 892ms
4335:	learn: 0.1028694	total: 23.4s	remaining:

RMSE = 259354.39, MAE = 139979.16, R-sq = 0.95, MAPE = 0.13 

RMSE = 256300.35, MAE = 146691.45, R-sq = 0.95, MAPE = 0.13 


In [331]:
y_pred = np.round((final_model.predict(stacked_features_test)/1000))*1000

sample_submission = pd.read_csv('sample_submission.csv')
sample_submission['price'] =  y_pred
sample_submission.to_csv(f'submission_stack_v1.csv', index=True)

sample_submission.head(10)

,id,price
0,0,1402000.0
1,1,2417000.0
2,2,999000.0
3,3,1914000.0
4,4,5136000.0
5,5,1445000.0
6,6,915000.0
7,7,582000.0
8,8,1079000.0
9,9,1241000.0
